In [1]:


from collections import defaultdict
import csv
import sys

import copy
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
import math
import pandas as pd
import matplotlib.pyplot
import random
import mxnet as mx
from datetime import datetime

%matplotlib inline

csv.field_size_limit(sys.maxsize);



In [2]:
DF = pd.read_csv('data/train_wkt_v4.csv')

In [3]:
ids = sorted(DF.ImageId.unique())
ids=['6040_2_2']

In [4]:
poly_types=range(1,11)

In [5]:
def addtwodimdict(thedict, key_a, key_b, val): 
    if key_a in thedict:
        thedict[key_a].update({key_b: val})
    else:
        thedict.update({key_a:{key_b: val}})

In [6]:
x_max={}
y_min={}
train_polygons={}
im_rgb={}
im_size={}

for IM_ID in ids:
    for POLY_TYPE in poly_types:
        # Load grid size
        for _im_id, _x, _y in csv.reader(open('data/grid_sizes.csv')):
            if _im_id == IM_ID:
                addtwodimdict(x_max, IM_ID, POLY_TYPE, float(_x))
                addtwodimdict(y_min, IM_ID, POLY_TYPE, float(_y))
                break

        # Load train poly with shapely
        for _im_id, _poly_type, _poly in csv.reader(open('data/train_wkt_v4.csv')):
            if _im_id == IM_ID and _poly_type == str(POLY_TYPE):
                addtwodimdict(train_polygons, IM_ID, POLY_TYPE, shapely.wkt.loads(_poly))
                break

        # Read image with tiff
        addtwodimdict(im_rgb, IM_ID, POLY_TYPE, tiff.imread('data/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0]))
        addtwodimdict(im_size, IM_ID, POLY_TYPE, im_rgb[IM_ID][POLY_TYPE].shape[:2])


In [7]:
train_polygons_scaled={}

def get_scalers(x_max,y_min,im_size):
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    w_ = w * (float(w) / (w + 1))
    h_ = h * (float(h) / (h + 1))
    return w_ / x_max, h_ / y_min

for IM_ID in ids:
    for POLY_TYPE in poly_types:
        x_scaler, y_scaler = get_scalers(x_max[IM_ID][POLY_TYPE],y_min[IM_ID][POLY_TYPE],im_size[IM_ID][POLY_TYPE])
        addtwodimdict(train_polygons_scaled, IM_ID, POLY_TYPE, shapely.affinity.scale(train_polygons[IM_ID][POLY_TYPE], xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0)))


In [8]:
train_mask={}

def mask_for_polygons(polygons,im_size):
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

for IM_ID in ids:
    for POLY_TYPE in poly_types:
        addtwodimdict(train_mask, IM_ID, POLY_TYPE, mask_for_polygons(train_polygons_scaled[IM_ID][POLY_TYPE],im_size[IM_ID][POLY_TYPE]))


In [9]:
# rotate(): rotate image  
# return: rotated image object  
def rotate(  
    img,  #image matrix  
    angle #angle of rotation  
    ):  
      
    height = img.shape[0]  
    width = img.shape[1]  
      
    if angle%180 == 0:  
        scale = 1  
    elif angle%90 == 0:  
        scale = float(max(height, width))/min(height, width)  
    else:  
        scale = math.sqrt(pow(height,2)+pow(width,2))/min(height, width)  
      
    #print 'scale %f\n' %scale  
          
    rotateMat = cv2.getRotationMatrix2D((width/2, height/2), angle, scale)  
    rotateImg = cv2.warpAffine(img, rotateMat, (int(width/scale), int(height/scale)) ) 
    #cv2.imshow('rotateImg',rotateImg)  
    #cv2.waitKey(0)  
      
    return rotateImg #rotated image 

In [10]:


def scale_percentile(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix



In [11]:
class SimpleIter:
    def __init__(self, num_batches=1280000000000000000):
        self.num_batches = num_batches
        self.cur_batch = 0

    def __iter__(self):
        return self

    def reset(self):
        self.cur_batch = 0        

    def __next__(self):
        return self.next()
    
    @property
    def provide_data(self):
        return zip(['data'],[(128,3,64,64)])

    @property
    def provide_label(self):
        return zip(['softmax_label'],[(128,256)])
    
    def next(self):
        temp_rgb={}
        temp_mask={}
        if random.randint(0,3)!=0:
            for anid in ids:
                temp_ang=random.randint(0,359)
                temp_rgb[anid]=rotate(im_rgb[anid][1],temp_ang)
                for m in range(1,11):
                    addtwodimdict(temp_mask, anid , m, rotate(train_mask[anid][m],temp_ang))
        else:
            for anid in ids:
                temp_rgb[anid]=im_rgb[anid][1]
                for m in range(1,11):
                    addtwodimdict(temp_mask, anid , m, train_mask[anid][m])
        if self.cur_batch < self.num_batches:
            data=[]
            label=[]
            self.cur_batch += 1
            for batch_num in range(0,128):   
                im_id=random.sample(ids,1)[0]
                x_samp=random.randint(0,temp_rgb[im_id].shape[0]-64)
                y_samp=random.randint(0,temp_rgb[im_id].shape[1]-64)
                temp_data = temp_rgb[im_id][:][x_samp:x_samp+64,y_samp:y_samp+64]
                temp_data = temp_data.transpose([2, 0, 1])
                data.append(temp_data)
                temp_label=[]
                for i in range(0,16):
                    for j in range(0,16):
                        bit=0
                        for area_type in range(1,11):
                            if int(temp_mask[im_id][area_type][x_samp+24+i][y_samp+24+j])==1:
                                bit=area_type
                        temp_label.append(bit)
                label.append(temp_label)
            return SimpleBatch(data, label)
        else:
            raise StopIteration

In [12]:
smpiter=SimpleIter()

In [13]:
class SimpleBatch(object):
    def __init__(self, data, label, pad=0):
        self.data = data
        self.label = label
        self.pad = pad

In [14]:
soft_name=[]
for l in range(0,256):
    soft_name.append('_'+str(l)+'_softmax_label')

In [15]:
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), name=None, suffix=''):
    conv = mx.symbol.Convolution(data=data, num_filter=num_filter, kernel=kernel, stride=stride, pad=pad, name='conv_%s%s' %(name, suffix))
    bn = mx.symbol.BatchNorm(data=conv, name='bn_%s%s' %(name, suffix))
    act = mx.symbol.Activation(data=bn, act_type='relu', name='relu_%s%s' %(name, suffix))
    return act


net = mx.sym.Variable("data")
net = ConvFactory(data=net, num_filter=64, kernel=(16,16), stride=(4,4), name='conv_1')
net = mx.sym.Pooling(data=net, pool_type="max", kernel=(2,2), stride=(1,1))
net = ConvFactory(data=net, num_filter=112, kernel=(4,4), stride=(1,1), name='conv_2')
net = ConvFactory(data=net, num_filter=80, kernel=(3,3), stride=(1,1), name='conv_3')
net = mx.sym.FullyConnected(data=net, name='fc1', num_hidden=4096)
net = mx.sym.Activation(data=net, name='relu1', act_type="relu")
# net = mx.sym.FullyConnected(data=net, num_hidden=11)
# net = mx.sym.SoftmaxOutput(data=net, name='softmax', multi_output=True)
soft_lay=[]
for j in range(0,256):
    temp_net = mx.sym.FullyConnected(data=net, num_hidden=11)
    soft_lay.append(mx.sym.SoftmaxOutput(data=temp_net, name='_'+str(j)+'_softmax'))
group = mx.symbol.Group(soft_lay)
# shape = {"data" : (128, 3, 64, 64)}
# mx.viz.plot_network(symbol=soft_lay[0], shape=shape)
#group.list_outputs()

In [16]:
mod = mx.mod.Module(symbol=group, 
                    context=mx.gpu(),
                    data_names=["data"], 
                    label_names=soft_name)

In [17]:
for li in range(0,1):
    for lops in range(0,1):
        test=smpiter.next()
        dt=mx.nd.array(test.data)
        temp_lbl =(np.array(test.label).transpose([1, 0]))
        lbl=[]
        for z in range(0,256):
            lbl.append(mx.nd.array(temp_lbl[z]))
        train_iter = mx.io.NDArrayIter(dt, lbl, 128, shuffle=True)
        a=datetime.now()
        mod.fit(train_iter, 
                eval_data=train_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.0005,'momentum':0.9,'wd':0.0002},
                eval_metric='acc',
                num_epoch=1)
    #     print (datetime.now()-a)
    print 'start line '+str(li) 
    for lops in range(0,3):
        test=smpiter.next()
        dt=mx.nd.array(test.data)
        temp_lbl =(np.array(test.label).transpose([1, 0]))
        lbl=[]
        for z in range(0,256):
            lbl.append(mx.nd.array(temp_lbl[z]))
        train_iter = mx.io.NDArrayIter(dt, lbl, 128, shuffle=True)
        print mod.score(train_iter, eval_metric='ce')
    mod.save_params('temp_line_'+str(li))

start line 0
[('cross-entropy', 2.384523562155664)]
[('cross-entropy', 2.385174029506743)]
[('cross-entropy', 2.3836285676807165)]


In [18]:
mod.load_params('goodpar1')

In [19]:
def pred_a_window(img):
    pred_iter = mx.io.NDArrayIter(mx.nd.array(img),batch_size=128)
    nouselab=mod.predict(pred_iter)
    output=[]
    for bits in nouselab:
        output.append(bits.asnumpy())
    return np.array(output).transpose([1,2,0])

In [20]:
def pred_with_mod(img):
    pred_list=[]
    pred_mask=[]
    for i in range(0,10):
        pred_mask.append(np.zeros(img.shape[1:]))
    for xs in range(0,img.shape[1]-64,16):
        for ys in range(0,img.shape[2]-64,16):
            pred_list.append(img[:,xs:xs+64,ys:ys+64])
        pred_list.append(img[:,xs:xs+64,img.shape[2]-64:img.shape[2]])
    xs=img.shape[1]-64
    for ys in range(0,img.shape[2]-64,16):
        pred_list.append(img[:,xs:xs+64,ys:ys+64])
    pred_list.append(img[:,xs:xs+64,img.shape[2]-64:img.shape[2]])
    output_list=pred_a_window(pred_list)
    
    count_list=0
    for xs in range(0,img.shape[1]-64,16):
        for ys in range(0,img.shape[2]-64,16):
            for i in range(0,10):
                pred_mask[i][xs+24:xs+40,ys+24:ys+40]=output_list[count_list][i+1].reshape(16,16)
            count_list += 1
        for i in range(0,10):
            pred_mask[i][xs+24:xs+40,img.shape[2]-40:img.shape[2]-24]=output_list[count_list][i+1].reshape(16,16)
        count_list += 1
    xs=img.shape[1]-64
    for ys in range(0,img.shape[2]-64,16):
        for i in range(0,10):
            pred_mask[i][xs+24:xs+40,ys+24:ys+40]=output_list[count_list][i+1].reshape(16,16)
        count_list += 1
    for i in range(0,10):
        pred_mask[i][xs+24:xs+40,img.shape[2]-40:img.shape[2]-24]=output_list[count_list][i+1].reshape(16,16)
    count_list += 1
    return pred_mask

In [21]:
# tiff.imshow(im_rgb[ids[11]][1])

In [22]:
# a=pred_with_mod(im_rgb[ids[11]][1].transpose([2,0,1]))

In [23]:
# tiff.imshow(train_mask[ids[11]][5])

In [24]:
def mask_to_polygons(mask, epsilon=8., min_area=5.):
    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

In [25]:
df = pd.read_csv('data/sample_submission.csv')
GS = pd.read_csv('data/grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
idss = sorted(df.ImageId.unique())
threshold = 0.1
temp_df=pd.DataFrame()
for oneid in idss:
    temp_df=pd.DataFrame()
    temp_im=tiff.imread('data/three_band/{}.tif'.format(oneid))
    temp_mask=pred_with_mod(temp_im)
    pred_binary_mask = np.array(temp_mask) >= threshold
    np.save('/media/yangzheliu/数据与工具/msk/10_%s' % oneid, pred_binary_mask)


for idx, row in df.iterrows():
    anid = row[0]
    kls = row[1] - 1

    msk = np.load('/media/yangzheliu/数据与工具/msk/10_%s.npy' % anid)[kls]
    pred_polygons = mask_to_polygons(msk)
    x_max = GS.loc[GS['ImageId'] == anid, 'Xmax'].as_matrix()[0]
    y_min = GS.loc[GS['ImageId'] == anid, 'Ymin'].as_matrix()[0]

    x_scaler, y_scaler = get_scalers(x_max, y_min, msk.shape)

    scaled_pred_polygons = shapely.affinity.scale(pred_polygons, xfact=1.0 / x_scaler, yfact=1.0 / y_scaler,
                                                      origin=(0, 0, 0))

    df.iloc[idx, 2] = shapely.wkt.dumps(scaled_pred_polygons)
    if idx % 100 == 0: print idx
df.to_csv('subm1.csv', index=False)

Self-intersection at or near point 3302.6034482758619 1461.6724137931035
Self-intersection at or near point 2915.0434782608695 2248.4347826086955


0


Self-intersection at or near point 3354.341232227488 2776.9810426540284
Self-intersection at or near point 3377.5684931506848 1669.3767123287671
Self-intersection at or near point 2470.7060133630289 2151.4922048997773
Self-intersection at or near point 2742.0344827586205 693.90344827586205
Self-intersection at or near point 798.08653846153845 2408.9634615384616
Self-intersection at or near point 1816.4968944099378 2408.1987577639752
Self-intersection at or near point 1975.2333333333333 2442.8083333333334
Self-intersection at or near point 3289.1379310344828 2131.9913793103447
Self-intersection at or near point 3360.7018633540374 1739.5279503105589
Self-intersection at or near point 3320 1000.0833333333334
Self-intersection at or near point 3143.2653658536587 439.01658536585364
Self-intersection at or near point 3322.1740139211138 1764.9466357308584
Self-intersection at or near point 3350.0491803278687 1871.2716627634661
Self-intersection at or near point 3335.344827586207 1639
Self-int

100


Self-intersection at or near point 3244.8916256157636 2105.6699507389162
Self-intersection at or near point 3114.6939130434785 418.54608695652172
Self-intersection at or near point 3368.7412587412587 2855.8951048951049
Self-intersection at or near point 3348.514469453376 1879.8938906752412
Self-intersection at or near point 3258.6557823129251 151.79455782312925
Self-intersection at or near point 3305.6774193548385 1901.1935483870968
Self-intersection at or near point 3227 1592
Self-intersection at or near point 3289.4827586206898 2675.5862068965516
Self-intersection at or near point 3250.5075125208682 836.5409015025042
Ring Self-intersection at or near point 3000 71
Self-intersection at or near point 3271.2473118279568 516.04301075268813
Self-intersection at or near point 2448.2758620689656 55
Self-intersection at or near point 3143.0138408304497 2472.1937716262978
Self-intersection at or near point 2793.0539906103286 164.01173708920189
Self-intersection at or near point 3042.364705882

...

Self-intersection at or near point 3026.7068965517242 2789.9310344827586
Self-intersection at or near point 3224.0999999999999 3287
Self-intersection at or near point 3221.6245674740485 3302.1453287197232
Self-intersection at or near point 2534.4615384615386 772.10769230769233
Self-intersection at or near point 2472 904
Self-intersection at or near point 3320.636815920398 1744.1144278606964
Self-intersection at or near point 3004.3824884792625 1373.0138248847927
Self-intersection at or near point 2267.1219512195121 1575.1951219512196
Self-intersection at or near point 3307.2838137472286 3266.4101995565411
Self-intersection at or near point 2888.8977119784658 2623.2786002691792
Self-intersection at or near point 3017.5826771653542 1372.0629921259842
Self-intersection at or near point 3368.1010101010102 1006.969696969697
Self-intersection at or near point 2719.8174273858922 1212.9792531120331
Self-intersection at or near point 3251.9908256880735 241.60550458715596
Self-intersection at or

4100


Self-intersection at or near point 3346 152
Self-intersection at or near point 3368.945355191257 3256.6448087431695
Self-intersection at or near point 3291.9473684210525 2606.3508771929824
Ring Self-intersection at or near point 2264 950
Self-intersection at or near point 2342.7752956636004 1088.7713534822601
Self-intersection at or near point 2101.6842105263158 282.22807017543857
Self-intersection at or near point 3208.5281690140846 629.21126760563379
Self-intersection at or near point 3351 440.33333333333331
Self-intersection at or near point 1061.7314487632509 3017.0070671378094
Self-intersection at or near point 3302.1875 1850.9479166666667
Self-intersection at or near point 3323.0168067226891 126.26050420168067
Self-intersection at or near point 3356.0766283524904 1928.5019157088122
Self-intersection at or near point 3208.1098265895953 2503.0693641618495
Self-intersection at or near point 3098.6341463414633 1322.8048780487804
Self-intersection at or near point 448.8235294117647 12

4200


Self-intersection at or near point 3176.9957081545062 2819.9442060085835
Self-intersection at or near point 3304.2279792746112 2121.7150259067357
Self-intersection at or near point 3351.5928143712576 2807.3532934131736
Self-intersection at or near point 3365.3493975903616 2199.9277108433735
Self-intersection at or near point 3197.6997084548107 84.967930029154516
Self-intersection at or near point 3347.3366733466933 1717.4549098196392
Self-intersection at or near point 3059.4020618556701 875.89690721649481
Self-intersection at or near point 3285.5194805194806 62.532467532467535
Self-intersection at or near point 2887.9677419354839 2392.0604838709678
Self-intersection at or near point 3346.3900293255133 381.55718475073314
Self-intersection at or near point 3345.2297447280798 1157.4250832408436
Self-intersection at or near point 3359.6710526315787 3271.7171052631579
Self-intersection at or near point 3117.2294117647057 2565.741176470588
Self-intersection at or near point 727.6631578947368